In [ ]:
%env SPEECH_KEY=<PUT_YOUR_API_KEY_HERE>
%env SPEECH_REGION=<PUT_YOUR_REGION_HERE>


In [3]:
import os
import azure.cognitiveservices.speech as speechsdk

# This example requires environment variables named "SPEECH_KEY" and "SPEECH_REGION"
speech_config = speechsdk.SpeechConfig(
    subscription=os.environ.get('SPEECH_KEY'), region=os.environ.get('SPEECH_REGION'))
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# The language of the voice that speaks.
speech_config.speech_synthesis_voice_name = 'en-US-JennyNeural'

speech_synthesizer = speechsdk.SpeechSynthesizer(
    speech_config=speech_config, audio_config=audio_config)

text = "Today is a cold and overcast day, I hope the sun comes out tomorrow."

speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized for text [{}]".format(text))
elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = speech_synthesis_result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(
                cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")


Enter some text that you want to speak >
Speech synthesis canceled: CancellationReason.Error
Error details: Connection failed (no connection to the remote host). Internal error: 1. Error details: Failed with error: WS_OPEN_ERROR_UNDERLYING_IO_OPEN_FAILED
wss://westeurope.tts.speech.microsoft.com/cognitiveservices/websocket/v1
X-ConnectionId: 7c08e2927ab1460095a2b8bef6dd28a5 USP state: Sending. Received audio size: 0 bytes.
Did you set the speech resource key and region values?


In [ ]:
ssml = """<speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis"
       xmlns:mstts="https://www.w3.org/2001/mstts" xml:lang="zh-CN">
    <voice name="en-US-EricNeural">
        The son runs over and says to his mom：
        <mstts:express-as role="Boy" style="cheerful">
            "Mom, can we get a dog?"
        </mstts:express-as>
    </voice>
    <voice name="en-US-AriaNeural">
        Mom points to her son's bedroom and says：
        <mstts:express-as role="SeniorFemale" style="angry">
            "Why, so we can have another creature in the house who eats and doesn't clean up?"
        </mstts:express-as>
    </voice>
</speak>"""

speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml).get()


In [ ]:
ssml = """<speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis"
       xmlns:mstts="https://www.w3.org/2001/mstts" xml:lang="en-US">
    <voice name="en-US-JennyNeural">
        <mstts:express-as style="hopeful">
            Just thinking about the upcoming family reunion fills my heart with a warm glow, as if all the joy in the world is waiting to embrace me.
        </mstts:express-as>
        <mstts:express-as style="advertisement_upbeat">
            Get ready to revolutionize your day with our ultra-energizing smoothie blend, a burst of flavor and vitality that will supercharge your mornings!
        </mstts:express-as>
    </voice>
</speak>"""

speech_synthesis_result = speech_synthesizer.speak_ssml_async(ssml).get()


In [ ]:
speech_config.speech_synthesis_language='en-US'
speech_config.speech_synthesis_voice_name='en-US-JennyNeural'

audio_config = speechsdk.audio.AudioOutputConfig(filename="./data/tts.wav")

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

text = "Today is a cold and overcast day, I hope the sun comes out tomorrow."
speech_synthesizer.speak_text_async(text)


In [ ]:
speech_config.set_speech_synthesis_output_format(speechsdk.SpeechSynthesisOutputFormat.Audio48Khz192KBitRateMonoMp3)

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
result = speech_synthesizer.speak_text_async(text).get()
stream =speechsdk.AudioDataStream(result)

stream.save_to_wav_file("./data/tts.mp3")


In [17]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]
model.cuda()
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator([model], cfg)

text = "Today is a cold and overcast day, I hope the sun comes out tomorrow."

sample = TTSHubInterface.get_model_input(task, text)
sample["net_input"]["src_tokens"] = sample["net_input"]["src_tokens"].cuda()
sample["net_input"]["src_lengths"] = sample["net_input"]["src_lengths"].cuda()

wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)
ipd.Audio(wav.cpu(), rate=rate)


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

2023-12-01 12:29:17 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75
2023-12-01 12:29:17 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/fw/.cache/fairseq/models--facebook--fastspeech2-en-ljspeech/snapshots/a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin
